## Spark

#### most popular movies imdb mk-100k u.item dataset

In [1]:
# Spark 2 code
# export SPARK_MAJOR_VERSION=2
# spark-submit LowestRatedMovieDataFrame.py


from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions

In [2]:
def loadMovieNames():
    movieNames = {}
    with open("ml-100k/u.item") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

In [3]:
def parseInput(line):
    fields = line.split()
    return Row(movieID = int(fields[1]), rating = float(fields[2]))


In [4]:
# Create a SparkSession (the config bit is only for Windows!)
spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

# Load up our movie ID -> name dictionary
movieNames = loadMovieNames()


In [5]:
# Get the raw data
lines = spark.sparkContext.textFile("file:///C:/Users/SanmiLee/Desktop/SanmiLeeAI/spark/ml-100k/u.data")

# Convert it to a RDD of Row objects with (movieID, rating)
movies = lines.map(parseInput)

# Convert that to a DataFrame
movieDataset = spark.createDataFrame(movies)


In [6]:

# Compute average rating for each movieID
averageRatings = movieDataset.groupBy("movieID").avg("rating")

# Compute count of ratings for each movieID
counts = movieDataset.groupBy("movieID").count()

# Join the two together (We now have movieID, avg(rating), and count columns)
averagesAndCounts = counts.join(averageRatings, "movieID")

# Pull the top 10 results
topTen = averagesAndCounts.orderBy("avg(rating)").take(10)


In [7]:

# Print them out, converting movie ID's to names as we go.
for movie in topTen:
    print (movieNames[movie[0]], movie[1], movie[2])


Amityville: A New Generation (1993) 5 1.0
Hostile Intentions (1994) 1 1.0
Lotto Land (1995) 1 1.0
Careful (1992) 1 1.0
Power 98 (1995) 1 1.0
Low Life, The (1994) 1 1.0
Further Gesture, A (1996) 1 1.0
Falling in Love Again (1980) 2 1.0
Amityville: Dollhouse (1996) 3 1.0
Touki Bouki (Journey of the Hyena) (1973) 1 1.0


In [8]:
# Stop the session
spark.stop()